## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,53.26,71,40,24.16,scattered clouds
1,1,Yellowknife,CA,62.4560,-114.3525,-17.88,76,90,4.61,light shower snow
2,2,Cherskiy,RU,68.7500,161.3000,-24.57,95,100,1.63,overcast clouds
3,3,Muros,ES,42.7762,-9.0603,51.24,94,23,5.73,few clouds
4,4,Dikson,RU,73.5069,80.5464,-23.64,94,92,6.62,overcast clouds


In [16]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [17]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Rikitea,PF,-23.1203,-134.9692,77.47,76,100,12.57,moderate rain
8,8,Avarua,CK,-21.2078,-159.7750,82.45,61,75,11.50,broken clouds
10,10,Tra Vinh,VN,9.9347,106.3453,79.70,59,100,11.25,overcast clouds
12,12,Tucuman,AR,-26.8241,-65.2226,78.80,73,0,3.44,clear sky
16,16,Butaritari,KI,3.0707,172.7902,82.31,79,72,16.31,broken clouds
17,17,Bathsheba,BB,13.2167,-59.5167,79.41,78,40,19.57,scattered clouds
18,18,Bengkulu,ID,-3.8004,102.2655,79.11,79,100,7.49,overcast clouds
26,26,Contamana,PE,-7.3333,-75.0167,81.81,67,65,3.47,broken clouds
28,28,Vaini,TO,-21.2000,-175.2000,89.76,79,20,9.22,light rain
29,29,Flinders,AU,-34.5833,150.8552,89.06,63,85,10.27,overcast clouds


In [18]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                167
City                   167
Country                167
Lat                    167
Lng                    167
Max Temp               167
Humidity               167
Cloudiness             167
Wind Speed             167
Current Description    167
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                167
City                   167
Country                167
Lat                    167
Lng                    167
Max Temp               167
Humidity               167
Cloudiness             167
Wind Speed             167
Current Description    167
dtype: int64

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Rikitea,PF,77.47,moderate rain,-23.1203,-134.9692,
8,Avarua,CK,82.45,broken clouds,-21.2078,-159.7750,
10,Tra Vinh,VN,79.70,overcast clouds,9.9347,106.3453,
12,Tucuman,AR,78.80,clear sky,-26.8241,-65.2226,
16,Butaritari,KI,82.31,broken clouds,3.0707,172.7902,
17,Bathsheba,BB,79.41,scattered clouds,13.2167,-59.5167,
18,Bengkulu,ID,79.11,overcast clouds,-3.8004,102.2655,
26,Contamana,PE,81.81,broken clouds,-7.3333,-75.0167,
28,Vaini,TO,89.76,light rain,-21.2000,-175.2000,
29,Flinders,AU,89.06,overcast clouds,-34.5833,150.8552,


In [64]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")  
   

In [65]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Rikitea,PF,77.47,moderate rain,-23.1203,-134.9692,People ThankYou
8,Avarua,CK,82.45,broken clouds,-21.2078,-159.7750,Paradise Inn
10,Tra Vinh,VN,79.70,overcast clouds,9.9347,106.3453,Hồng Ân
12,Tucuman,AR,78.80,clear sky,-26.8241,-65.2226,Dallas Hotel Tucuman
16,Butaritari,KI,82.31,broken clouds,3.0707,172.7902,Isles Sunset Lodge
...,...,...,...,...,...,...,...
677,Tierralta,CO,76.60,overcast clouds,8.1736,-76.0592,
684,Benguela,AO,75.27,overcast clouds,-12.5763,13.4055,
686,Labuhan,ID,84.72,overcast clouds,-6.8844,112.2051,
689,Muli,MV,81.55,overcast clouds,2.9167,73.5667,


In [77]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.eq("").sum()
hotel_df.replace("", "NaN")



,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Rikitea,PF,77.47,moderate rain,-23.1203,-134.9692,People ThankYou
8,Avarua,CK,82.45,broken clouds,-21.2078,-159.7750,Paradise Inn
10,Tra Vinh,VN,79.70,overcast clouds,9.9347,106.3453,Hồng Ân
12,Tucuman,AR,78.80,clear sky,-26.8241,-65.2226,Dallas Hotel Tucuman
16,Butaritari,KI,82.31,broken clouds,3.0707,172.7902,Isles Sunset Lodge
...,...,...,...,...,...,...,...
677,Tierralta,CO,76.60,overcast clouds,8.1736,-76.0592,NaN
684,Benguela,AO,75.27,overcast clouds,-12.5763,13.4055,NaN
686,Labuhan,ID,84.72,overcast clouds,-6.8844,112.2051,NaN
689,Muli,MV,81.55,overcast clouds,2.9167,73.5667,NaN


In [78]:
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Rikitea,PF,77.47,moderate rain,-23.1203,-134.9692,People ThankYou
8,Avarua,CK,82.45,broken clouds,-21.2078,-159.7750,Paradise Inn
10,Tra Vinh,VN,79.70,overcast clouds,9.9347,106.3453,Hồng Ân
12,Tucuman,AR,78.80,clear sky,-26.8241,-65.2226,Dallas Hotel Tucuman
16,Butaritari,KI,82.31,broken clouds,3.0707,172.7902,Isles Sunset Lodge
...,...,...,...,...,...,...,...
677,Tierralta,CO,76.60,overcast clouds,8.1736,-76.0592,
684,Benguela,AO,75.27,overcast clouds,-12.5763,13.4055,
686,Labuhan,ID,84.72,overcast clouds,-6.8844,112.2051,
689,Muli,MV,81.55,overcast clouds,2.9167,73.5667,


In [67]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [57]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [58]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))